In [11]:
from sail_safe_functions_test.helper_sail_safe_functions.series_federated_local import SeriesFederatedLocal
from sail_safe_functions_test.helper_sail_safe_functions.tools_test_data import ToolsTestData

import plotly.graph_objects as go
import numpy
import pandas
import os


import sail_safe_functions_orchestrator.preprocessing
import sail_safe_functions_orchestrator.statistics
from sail_safe_functions_orchestrator.statistics.kolmogorov_smirnov_test_federate import KolmogorovSmirnovTestFederate
from sail_safe_functions_orchestrator.statistics.mann_whitney_u_test_federate import MannWhitneyUTestFederate
from sail_safe_functions_orchestrator.statistics.wilcoxon_singed_rank_test_federate import WilcoxonSingedRankTestFederate


from sail_safe_functions_orchestrator.statistics.mean_federate import MeanFederate
from sail_safe_functions_orchestrator.statistics.variance_federate import VarianceFederate
from sail_safe_functions_orchestrator.statistics.skewness_federate import SkewnessFederate
from sail_safe_functions_orchestrator.statistics.kurtosis_federate import KurtosisFederate


from sail_safe_functions_orchestrator.statistics.pearson_federate import PearsonFederate
from sail_safe_functions_orchestrator.statistics.spearman_federate import SpearmanFederate
from sail_safe_functions_orchestrator.preprocessing.cdf_federate import CdfFederate
from sail_safe_functions_orchestrator.statistics.min_max_federate import MinMaxFederate


from sail_safe_functions_orchestrator.statistics.unpaired_t_test_federate import UnpairedTTestFederate
from sail_safe_functions_orchestrator.statistics.paired_t_test_federate import PairedTTestFederate

In [2]:
name_dataset = "data_csv_kidney_clean"
name_table = "1"
print (os.environ.get("PATH_SAIL_TEST_DATA"))
data_frame = ToolsTestData.load_test_dataframe_csv(name_dataset, name_table)



/home/jaap/datascience/sail-safe-functions-test/sail_safe_functions_test/data_sail_safe_functions/


In [3]:
sample_age = data_frame["age"]
alternative = "two-sided"
sample_blood_presure = data_frame["bp"]
sample_normal = SeriesFederatedLocal()
sample_normal_shifted = SeriesFederatedLocal()
sample_normal_paired = SeriesFederatedLocal()
array_normal = pandas.Series(numpy.random.normal(0, 1, 400))
array_normal_shifted =  pandas.Series(numpy.random.normal(0.1, 1, 400))
array_normal_paired = array_normal + 0.1 + pandas.Series(numpy.random.normal(0, 1, 400))
     
sample_normal.add_series("dataset_0", array_normal)
sample_normal_shifted.add_series("dataset_0", array_normal_shifted)
sample_normal_paired.add_series("dataset_0", array_normal_paired)

In [10]:
def compare_cdf(sample, title):
    list_domain, list_value = preprocessing.cdf.run(sample)
    list_domain_ref, list_value_ref = CdfFederate.run_reference(sample)
    line_0 = go.Scatter(
        x = list_domain, 
        y = list_value,
        mode = 'lines',
        name = 'safe',
    )

    line_1 = go.Scatter(
        x = list_domain_ref, 
        y = list_value_ref,
        mode = 'lines',
        name = 'reference',
    )
    fig = go.Figure([line_0, line_1])
    fig.update_layout(title=title)
    fig.show()

compare_cdf(sample_age, "age")
compare_cdf(sample_blood_presure, "blood_presure")
compare_cdf(sample_normal, "normal")
compare_cdf(sample_normal_shifted, "normal_shifted")

NameError: name 'preprocessing' is not defined

In [5]:
print(f"Basic statistics")
print(f"Count:     {sample_age.size}")
print(f"Min        {MinMaxFederate.run(sample_age)[0]}")
print(f"Max        {MinMaxFederate.run(sample_age)[1]}")
print(f"Mean:      {MeanFederate.mean(sample_age)}")
print(f"Variance:  {VarianceFederate.run(sample_age)}")
print(f"Skewness:  {SkewnessFederate.Run(sample_age)}")
print(f"Kurtosis:  {KurtosisFederate.Run(sample_age)}")
print()

Basic statistics
Count:     400
Min        1.2774999999999999
Max        90.5625
Mean:      51.51180500876386
Variance:  288.30719232447746
Skewness:  -0.6777244023262959
Kurtosis:  0.1129562132795261



In [6]:
print(f"KolmogorovSmirnov")
print(f"not normal is normal: {0.05 < KolmogorovSmirnovTestFederate.run(sample_age, 'normalunit', 'cdf')[1]}")
print(f"normal is normal:     {0.05 < KolmogorovSmirnovTestFederate.run(sample_normal, 'normalunit', 'cdf')[1]}")
print()

KolmogorovSmirnov
not normal is normal: False
normal is normal:     True



In [7]:
print(f"Spearman correlation")
print(f"correlated is uncorrelated:    {0.05 < SpearmanFederate.run(sample_age, sample_blood_presure, alternative, 'cdf')[1]}")
print(f"Yes, this is wrong, because the federated spearman cdf aproximation does not work well on low resolution data.")
print(f"correlated is uncorrelated:    {0.05 < SpearmanFederate.run(sample_age, sample_blood_presure, alternative, 'unsafe')[1]}")
print(f"uncorrelated is uncorrelated:  {0.05 < PearsonFederate.run(sample_normal, sample_normal_shifted)[1]}")
print()

Spearman correlation
correlated is uncorrelated:    True
Yes, this is wrong, because the federated spearman cdf aproximation does not work well on low resolution data.
correlated is uncorrelated:    False
uncorrelated is uncorrelated:  True



In [8]:
print(f" Paired equal means are different:     {0.05 < PairedTTestFederate.ttest_rel(sample_age, sample_age, alternative=alternative)[1]}")
print(f" Paired different means are different: {0.05 < PairedTTestFederate.ttest_rel(sample_normal, sample_normal_paired, alternative=alternative)[1]}")

print(f" Unpaired equal means are different:     {0.95 < UnpairedTTestFederate.ttest_ind(sample_age, sample_age, alternative=alternative)[1]}")
print(f" Unpaired different means are different: {0.05 < UnpairedTTestFederate.ttest_ind(sample_normal, sample_normal_paired, alternative=alternative)[1]}")


 Paired equal means are different:     False
 Paired different means are different: False
 Unpaired equal means are different:     False
 Unpaired different means are different: True


/home/jaap/datascience/sail-safe-functions/sail_safe_functions/statistics/paired_t_test_agregate.py:33: RuntimeWarning:

invalid value encountered in double_scalars

